In [4]:
import sys
from pathlib import Path

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

IN_DIR = DIR/'examples'/'inputs'
OUT_DIR = DIR/'examples'/'outputs'

%load_ext autoreload
%autoreload 2

high_definition = False


In [ ]:
# To test high definition download use
# http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N00E006.SRTMGL1.hgt.zip
# username='bingobongo', password='Bingo0Bongo'

In [ ]:
# Download elevation data

bounds = [174, -37, 176, -36]
names = wt.get_srtm_tile_names(bounds)
print('Tiles to download =', names)


if high_definition:
    path = IN_DIR/'elevations-hd'
else:
    path = IN_DIR/'elevations-sd'
    
wt.download_elevation_data_nasa(bounds, path, high_definition=high_definition,
  username='bingobongo', password='Bingo0Bongo')

In [ ]:
# Create SPLAT transmitters data

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
wt.create_splat_transmitter_data(in_path, out_path)
% ls {out_path}

In [ ]:
# Create SPLAT elevation data

if high_definition:
    in_path = IN_DIR/'elevations-hd'
else:
    in_path = IN_DIR/'elevations-sd'
out_path = OUT_DIR/'splat_files'

wt.create_splat_elevation_data(in_path, out_path, high_definition=high_definition)
% ls {out_path}

In [6]:
# Create a coverage maps

transmitter_names = ['MyNetwork_11']
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR
wt.create_coverage_maps(in_path, out_path, transmitter_names, high_definition=high_definition)
% ls {out_path/transmitter_names[0]}

Timing create_coverage_maps...
2016-08-08 13:46:19.936088   Began process
2016-08-08 13:49:45.472541   Finished in 3.42 min
MyNetwork_11-ck.ppm           MyNetwork_11.kml
MyNetwork_11-site_report.txt
